In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time

import sys

from pathlib import Path

%matplotlib notebook

In [ ]:
probe = model.define_probe(probe_type="linear")

In [ ]:
l5pc_cell = model.create()

param_names = [param.name for param in l5pc_cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(probe, protocols_with_lfp=['Step1'])

fitness_calculator = evaluator.define_fitness_calculator(
    fitness_protocols,
    feature_file=Path("./config/features.json"),
    feature_set='multiple'
)

sim = ephys.simulators.LFPySimulator(l5pc_cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=l5pc_cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
fitness_protocols

In [ ]:
release_params = evaluator.get_release_params()

In [ ]:
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values=release_params)

In [ ]:
t_start = time.time()
resp_step1 = fitness_protocols['Step1'].run(cell_model=l5pc_cell, param_values=release_params, sim=sim)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
fitness_protocols["Step1"].recordings

In [ ]:
responses.keys()

In [ ]:
def filter_lfp_response(response, fs=20.0, fcut=[100, 3000], order=2, filt_type="filtfilt"):
    from utils import _interpolate_response, _filter_response
    response_new = _interpolate_response(response, fs=fs)
    response_new = _filter_response(response_new, fcut=fcut, order=order, filt_type=filt_type)
    return response_new

In [ ]:
response_unfilt = responses['Step1.MEA.LFP']
repsonse_filt = filter_lfp_response(response_unfilt)

In [ ]:
plt.figure()
_ = plt.plot(repsonse_filt['time'], repsonse_filt['voltage'][5:7].T, color='C0')
_ = plt.plot(response_unfilt['time'], response_unfilt['voltage'][5:7].T, color='C1')

In [ ]:
plt.figure()
_ = plt.plot(responses['bAP.soma.v']['time'], responses['bAP.soma.v']['voltage'].T)
_ = plt.plot(responses['bAP.dend1.v']['time'], responses['bAP.dend1.v']['voltage'].T)
_ = plt.plot(responses['bAP.dend2.v']['time'], responses['bAP.dend2.v']['voltage'].T)

In [ ]:
fitness_protocols

In [ ]:
loc = fitness_protocols['EPSP'].stimuli[0].location
type(loc)

In [ ]:
loc.soma_distance

In [ ]:
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})

In [ ]:
fig, axes = plt.subplots(len(responses), figsize=(10, 20), squeeze=False)
for index, (resp_name, response) in enumerate(sorted(responses.items())):
    axes[index, 0].plot(response['time'], response['voltage'], label=resp_name)
    axes[index, 0].set_title(resp_name)
fig.tight_layout()
fig.show()